# Task VI (b)
 Take the pre-trained model from Task VI.A and fine-tune it for a super-resolution task. The model should be fine-tuned to upscale low-resolution strong lensing images using the provided high-resolution samples as ground truths. Please implement your approach in PyTorch or Keras and discuss your strategy.


## Breakdown to approach:
* Our pre-trained model has already learned representations of lensing images from TaskVI (a). Instead of learning from scratch, this method refines learned knowledge and adapts it to a different but related task  (super-resolution).
* Upsampling (Transpose Convolutions + PixelShuffle): Enhances resolution while maintaining structural integrity. Final Refinement (Tanh Activation): Ensures smoother HR reconstructions.
* Domain specific Feature Extraction.
* L1 Loss: Reduces blurriness compared to MSE, improving fine detail reconstruction. AdamW Optimizer: Stabilizes training and prevents overfitting with weight decay.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
from PIL import Image


Model loading and prep

In [ ]:
# Load the pre-trained ResNet classifier model
model_path = "resnet_classifier.pth"
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Remove 'resnet.' prefix if present
new_state_dict = {k.replace("resnet.", ""): v for k, v in state_dict.items()}

# Remove classifier layer keys from state_dict
new_state_dict = {k: v for k, v in new_state_dict.items() if not k.startswith("fc.")}

pretrained_model = resnet18(pretrained=False)
pretrained_model.load_state_dict(new_state_dict, strict=False)


In [ ]:
# Modify the model for super-resolution
class SuperResolutionResNet(nn.Module):
    def __init__(self, base_model):
        super(SuperResolutionResNet, self).__init__()

        # Modify input layer to accept 1-channel grayscale images
        base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Extract convolutional layers only (remove fully connected layer)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])

        # Upsampling module for super-resolution
        self.upsample = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = F.interpolate(x, scale_factor=4, mode='bilinear', align_corners=False)
        x = self.upsample(x)
        return x

# Instantiate the model
model = SuperResolutionResNet(pretrained_model)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Lets understand Model

SuperResolutionResNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os

In [ ]:
# Load pre-trained ResNet classifier model
model_path = "resnet_classifier.pth"
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Remove 'resnet.' prefix
new_state_dict = {k.replace("resnet.", ""): v for k, v in state_dict.items()}
# Remove classifier layer keys
new_state_dict = {k: v for k, v in new_state_dict.items() if not k.startswith("fc.")}

# Load modified ResNet
pretrained_model = resnet18(pretrained=False)
pretrained_model.load_state_dict(new_state_dict, strict=False)


In [ ]:
# Super-Resolution Model using Transpose Convolutions and PixelShuffle
class SuperResolutionResNet(nn.Module):
    def __init__(self, base_model):
        super(SuperResolutionResNet, self).__init__()

        # Modify input layer to accept grayscale images
        base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Extract convolutional layers
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])

        # Upsampling module
        self.upsample = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 4, kernel_size=3, stride=1, padding=1),  # 4 channels for pixel shuffle
            nn.PixelShuffle(2),  # Upscales by 2x
            nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),  # Final refinement layer
            nn.Tanh()
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.upsample(x)
        return x


In [ ]:
# Initialize model
model = SuperResolutionResNet(pretrained_model)

# Freeze early layers and fine-tune only later layers
for param in model.feature_extractor[:6].parameters():
    param.requires_grad = False

# Define L1 Loss and AdamW optimizer
criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Custom Dataset Class
class SuperResolutionDataset(Dataset):
    def __init__(self, dataset_path):
        self.lr_path = os.path.join(dataset_path, "LR")
        self.hr_path = os.path.join(dataset_path, "HR")
        self.lr_files = sorted(os.listdir(self.lr_path))
        self.hr_files = sorted(os.listdir(self.hr_path))

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = np.load(os.path.join(self.lr_path, self.lr_files[idx]))
        hr_image = np.load(os.path.join(self.hr_path, self.hr_files[idx]))

        # Ensure correct dimensions
        lr_image = np.expand_dims(lr_image, axis=0) if len(lr_image.shape) == 2 else lr_image
        hr_image = np.expand_dims(hr_image, axis=0) if len(hr_image.shape) == 2 else hr_image

        lr_image = torch.tensor(lr_image, dtype=torch.float32)
        hr_image = torch.tensor(hr_image, dtype=torch.float32)

        return lr_image, hr_image

# Load Dataset
dataset_path = "Dataset2/Dataset"
dataset = SuperResolutionDataset(dataset_path)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Training Loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for lr_images, hr_images in train_loader:
            lr_images, hr_images = lr_images.to(device), hr_images.to(device)
            optimizer.zero_grad()
            outputs = model(lr_images)

            # Ensure output size matches target size
            outputs = F.interpolate(outputs, size=hr_images.shape[2:], mode='bilinear', align_corners=False)

            loss = criterion(outputs, hr_images)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.6f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=20)


Epoch [1/20], Loss: 0.040184
Epoch [2/20], Loss: 0.025419
Epoch [3/20], Loss: 0.022257
Epoch [4/20], Loss: 0.019837
Epoch [5/20], Loss: 0.018492
Epoch [6/20], Loss: 0.018193
Epoch [7/20], Loss: 0.017289
Epoch [8/20], Loss: 0.016779
Epoch [9/20], Loss: 0.016173
Epoch [10/20], Loss: 0.016080
Epoch [11/20], Loss: 0.015976
Epoch [12/20], Loss: 0.015278
Epoch [13/20], Loss: 0.015062
Epoch [14/20], Loss: 0.014190
Epoch [15/20], Loss: 0.013255
Epoch [16/20], Loss: 0.012234
Epoch [17/20], Loss: 0.011827
Epoch [18/20], Loss: 0.011456
Epoch [19/20], Loss: 0.011448
Epoch [20/20], Loss: 0.011247


In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    avg_psnr, avg_ssim, avg_mse = 0, 0, 0
    with torch.no_grad():
        for lr_images, hr_images in test_loader:
            lr_images, hr_images = lr_images.to(device), hr_images.to(device)
            outputs = model(lr_images)
            outputs = F.interpolate(outputs, size=hr_images.shape[2:], mode='bilinear', align_corners=False)

            outputs = outputs.cpu().numpy()
            hr_images = hr_images.cpu().numpy()

            for i in range(len(outputs)):
                avg_psnr += psnr(hr_images[i, 0], outputs[i, 0], data_range=1.0)
                avg_ssim += ssim(hr_images[i, 0], outputs[i, 0], data_range=1.0)
                avg_mse += np.mean((hr_images[i, 0] - outputs[i, 0]) ** 2)

    avg_psnr /= len(test_loader.dataset)
    avg_ssim /= len(test_loader.dataset)
    avg_mse /= len(test_loader.dataset)
    print(f"Average PSNR: {avg_psnr:.2f} dB, Average SSIM: {avg_ssim:.4f}, Average MSE: {avg_mse:.6f}")

# Evaluate the model
evaluate_model(model, test_loader)


Average PSNR: 31.47 dB, Average SSIM: 0.9310, Average MSE: 0.000736
